#IMPORTING LIBRARIES

In [78]:
import pandas as pd
import numpy as np
import re,string
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
import gensim.downloader as api
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#IMPORTING DATA

In [81]:
data=pd.read_csv("/content/drive/MyDrive/IMDB/IMDB Dataset.csv",header=0)
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


#UNDERSTANDING THE DATA

In [82]:
data.isna().sum()

review       0
sentiment    0
dtype: int64

In [83]:
for i in range(data.shape[1]):
  print(data.iloc[:,i].unique())

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [84]:
for i in range(data.shape[1]):
  print(data.iloc[:,i].value_counts())

Loved today's show!!! It was a variety and not solely cooking (which would have been great too). Very stimulating and captivating, always keeping the viewer peeking around the corner to see what was coming up next. She is as down to earth and as personable as you get, like one of us which made the show all the more enjoyable. Special guests, who are friends as well made for a nice surprise too. Loved the 'first' theme and that the audience was invited to play along too. I must admit I was shocked to see her come in under her time limits on a few things, but she did it and by golly I'll be writing those recipes down. Saving time in the kitchen means more time with family. Those who haven't tuned in yet, find out what channel and the time, I assure you that you won't be disappointed.                                                                                                                                                                                                                

In [85]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [86]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [87]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

#Preprocessing

In [88]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [89]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
data['review']=data['review'].apply(denoise_text)

##SPECIAL CHARACTERS

In [90]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
data['review']=data['review'].apply(remove_special_characters)

##STEMMING

In [91]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
data['review']=data['review'].apply(simple_stemmer)

In [92]:
data

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl product the film techniqu is ve...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there a famili where a littl boy jake th...,negative
4,petter mattei love in the time of money is a v...,positive
...,...,...
49995,i thought thi movi did a down right good job i...,positive
49996,bad plot bad dialogu bad act idiot direct the ...,negative
49997,i am a cathol taught in parochi elementari sch...,negative
49998,im go to have to disagre with the previou comm...,negative


##STOPWORDS

In [93]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
data['review']=data['review'].apply(remove_stopwords)

{"didn't", 'was', 'between', 'those', 'there', 'before', 'did', 'couldn', 'me', 'same', 'do', "you'll", 'after', 'until', 'can', "hadn't", 'won', "you'd", 'o', 'but', 'other', 'ma', 'down', 'if', 'once', "you've", 'by', 'her', 'shan', 'out', "mustn't", 'these', 'himself', 'while', 'into', 'being', 'each', 'and', 'most', 'haven', 'it', 'be', "she's", 'not', 'more', 'your', 'does', "mightn't", "shouldn't", 'again', 'were', 'theirs', 'what', "aren't", 'needn', "shan't", 'should', 'who', 'whom', 'or', 'then', 'herself', 'because', 'where', "haven't", "wouldn't", 'over', "weren't", 'their', 'when', 'd', "wasn't", 'the', 'doesn', 'with', 'nor', 'don', 'an', 'now', 're', 'at', 'all', 'y', 'through', "doesn't", 'have', 'about', "should've", 'doing', 'didn', 'off', 'ourselves', 'ours', 'some', 'as', 'against', 'wasn', "needn't", 'has', 'yourself', 'in', "isn't", 'themselves', 'she', 'very', 'of', 'above', 'him', 'own', 'aren', 'ain', 'am', 'a', 'mightn', 'weren', 'are', "hasn't", 'than', 'any',

In [94]:
data

,review,sentiment
0,one review ha mention watch 1 oz episod youll ...,positive
1,wonder littl product film techniqu veri unassu...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic famili littl boy jake think zombi hi clo...,negative
4,petter mattei love time money visual stun film...,positive
...,...,...
49995,thought thi movi right good job wasnt creativ ...,positive
49996,bad plot bad dialogu bad act idiot direct anno...,negative
49997,cathol taught parochi elementari school nun ta...,negative
49998,im go disagre previou comment side maltin thi ...,negative


#TFIDF

In [95]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

nlp = spacy.load("en_core_web_sm")

In [96]:
X= data['review']
y=data["sentiment"]
X.head

<bound method NDFrame.head of 0        one review ha mention watch 1 oz episod youll ...
1        wonder littl product film techniqu veri unassu...
2        thought thi wa wonder way spend time hot summe...
3        basic famili littl boy jake think zombi hi clo...
4        petter mattei love time money visual stun film...
                               ...                        
49995    thought thi movi right good job wasnt creativ ...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    cathol taught parochi elementari school nun ta...
49998    im go disagre previou comment side maltin thi ...
49999    one expect star trek movi high art fan expect ...
Name: review, Length: 50000, dtype: object>

In [97]:
print("\n\nWith TFIDFVectorizer")
vectorizer = TfidfVectorizer(max_features=10)
X = vectorizer.fit_transform(data.review)
print(vectorizer.get_feature_names_out())
print(X.toarray())
print("\n")
#print(cosine_similarity(X))



With TFIDFVectorizer
['film' 'good' 'ha' 'hi' 'like' 'movi' 'one' 'thi' 'time' 'wa']
[[0.         0.         0.3202831  ... 0.55433648 0.         0.7217685 ]
 [0.31971564 0.         0.79968461 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.45204206 0.2637984  0.78476937]
 ...
 [0.         0.41513474 0.40170614 ... 0.69526107 0.         0.30175267]
 [0.14756947 0.         0.         ... 0.31941907 0.18640353 0.27726435]
 [0.         0.12295239 0.         ... 0.41183743 0.         0.        ]]




In [98]:
print("\n\nWith TFIDFVectorizer and removing stop words")
vectorizer = TfidfVectorizer(stop_words=nlp.Defaults.stop_words,max_features=10)
X = vectorizer.fit_transform(data.review)
print(vectorizer.get_feature_names_out())
print(X.toarray())
print("\n")
#print(cosine_similarity(X))



With TFIDFVectorizer and removing stop words


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


['charact' 'film' 'good' 'ha' 'hi' 'like' 'movi' 'thi' 'time' 'wa']
[[0.         0.         0.         ... 0.57456692 0.         0.74810934]
 [0.         0.33848318 0.         ... 0.         0.         0.        ]
 [0.2838305  0.         0.         ... 0.44377949 0.2589766  0.7704251 ]
 ...
 [0.         0.         0.41513474 ... 0.69526107 0.         0.30175267]
 [0.         0.18555452 0.         ... 0.40163897 0.23438464 0.34863345]
 [0.13118999 0.         0.12247573 ... 0.41024081 0.         0.        ]]




In [99]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##Logistic Regression

In [100]:
# Create a logistic regression object
lr1 = LogisticRegression()
lr1.fit(X_train, y_train)
y_pred = lr1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 57.873333333333335


##KNN

In [101]:
# Create a KNN
from sklearn.neighbors import KNeighborsClassifier
model1=KNeighborsClassifier(n_neighbors=5)
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 54.35333333333333


##SVM

In [102]:
# Create a SVM
from sklearn import svm
cls1=svm.SVC(kernel='rbf')
cls1.fit(X_train, y_train)
y_pred = cls1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 58.34


##Bagging Classifier

In [103]:
# Create a Bagging classifier
from sklearn.ensemble import BaggingClassifier
clf1 = BaggingClassifier()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 54.32666666666667


##Decision Tree

In [104]:
# Create a Decision Tree
from sklearn.tree import DecisionTreeClassifier
tree1 = DecisionTreeClassifier()
tree1.fit(X_train, y_train)
y_pred = tree1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 52.38


##Random Forest

In [105]:
# Create a Random Forest
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_train, y_train)
y_pred = rf1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 56.49999999999999


##Ada Boost

In [106]:
# Create a Ada Boost
from sklearn.ensemble import AdaBoostClassifier
adaboost1 = AdaBoostClassifier()
adaboost1.fit(X_train, y_train)
y_pred = adaboost1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 57.89333333333333


##Gradient Boost

In [107]:
# Create a Gradient Boost
from sklearn.ensemble import GradientBoostingClassifier
grad_boost1 = GradientBoostingClassifier()
grad_boost1.fit(X_train, y_train)
y_pred = grad_boost1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 57.82000000000001


##XG Boost

In [108]:
# Create a XG Boost
import xgboost as xgb
from xgboost import XGBClassifier
xgb_boost1 = XGBClassifier()
xgb_boost1.fit(X_train, y_train)
y_pred = xgb_boost1.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 58.08


#Word2vec

In [109]:
# Train and download Word2Vec vectors
sentences = [review.split() for review in data['review']]
model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)
model.save('word2vec.model')

In [110]:
# Convert each word to vector and represent the sentence in vector form using the word embeddings
def sentence_vector(sentence,model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv.vocab]
    if len(word_vectors) == 0:
        return np.zeros((100,))
    return np.mean(word_vectors, axis=0)

In [111]:
# Convert each sentence in the dataset to a vector using the Word2Vec model
word2vec_train = np.array([sentence_vector(sentence, model) for sentence in data['review']])
word2vec_test = np.array(data['sentiment'])

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(word2vec_train, word2vec_test, test_size=0.3, random_state=42)

##Logistic Regression

In [122]:
# Create a logistic regression object
lr2 = LogisticRegression()
lr2.fit(X_train, y_train)
y_pred = lr2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 85.37333333333333


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##KNN

In [114]:
# Create a KNN
from sklearn.neighbors import KNeighborsClassifier
model2=KNeighborsClassifier(n_neighbors=5)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 79.56666666666666


##SVM

In [115]:
# Create a SVM
from sklearn import svm
cls2=svm.SVC(kernel='rbf')
cls2.fit(X_train, y_train)
y_pred = cls2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 85.79333333333334


##Bagging Classifier

In [116]:
# Create a Bagging classifier
from sklearn.ensemble import BaggingClassifier
clf2 = BaggingClassifier()
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 79.37333333333333


##Decision Tree

In [117]:
# Create a Decision Tree
from sklearn.tree import DecisionTreeClassifier
tree2 = DecisionTreeClassifier()
tree2.fit(X_train, y_train)
y_pred = tree2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 72.88666666666667


##Random Forest

In [118]:
# Create a Random Forest
from sklearn.ensemble import RandomForestClassifier
rf2 = RandomForestClassifier()
rf2.fit(X_train, y_train)
y_pred = rf2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 83.40666666666667


##Ada Boost

In [119]:
# Create a Ada Boost
from sklearn.ensemble import AdaBoostClassifier
adaboost2 = AdaBoostClassifier()
adaboost2.fit(X_train, y_train)
y_pred = adaboost2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 82.04666666666667


##Gradient Boost

In [120]:
# Create a Gradient Boost
from sklearn.ensemble import GradientBoostingClassifier
grad_boost2 = GradientBoostingClassifier()
grad_boost2.fit(X_train, y_train)
y_pred = grad_boost2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 83.71333333333332


##XG Boost

In [121]:
# Create a XG Boost
import xgboost as xgb
from xgboost import XGBClassifier
xgb_boost2 = xgb.XGBClassifier()
xgb_boost2.fit(X_train, y_train)
y_pred = xgb_boost2.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 83.76


#Combining tfidf and word2vec

In [123]:
def combine_vectors(doc):
    tfidf_vec = vectorizer.transform([doc])
    w2v_vec = sentence_vector(doc, model)
    combined_vec = np.concatenate([np.squeeze(tfidf_vec.toarray()), w2v_vec])
    return combined_vec

In [124]:
combined_train = np.array([combine_vectors(doc) for doc in data['review']])
combined_test = np.array(data['sentiment'])

In [125]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_train, combined_test, test_size=0.3, random_state=42)

##Logistic Regression

In [126]:
# Create a logistic regression object
lr3 = LogisticRegression()
lr3.fit(X_train, y_train)
y_pred = lr3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 85.30666666666666


##KNN

In [127]:
# Create a KNN
from sklearn.neighbors import KNeighborsClassifier
model3=KNeighborsClassifier(n_neighbors=5)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 78.81333333333333


##SVM

In [128]:
# Create a SVM
from sklearn import svm
cls3=svm.SVC(kernel='rbf')
cls3.fit(X_train, y_train)
y_pred = cls3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 85.90666666666667


##Bagging Classifier

In [129]:
# Create a Bagging classifier
from sklearn.ensemble import BaggingClassifier
clf3 = BaggingClassifier()
clf3.fit(X_train, y_train)
y_pred = clf3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 79.69333333333334


##Decision Tree

In [130]:
# Create a Decision Tree
from sklearn.tree import DecisionTreeClassifier
tree3 = DecisionTreeClassifier()
tree3.fit(X_train, y_train)
y_pred = tree3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 72.35333333333334


##Random Forest

In [131]:
# Create a Random Forest
from sklearn.ensemble import RandomForestClassifier
rf3 = RandomForestClassifier()
rf3.fit(X_train, y_train)
y_pred = rf3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 83.29333333333334


##Ada Boost

In [132]:
# Create a Ada Boost
from sklearn.ensemble import AdaBoostClassifier
adaboost3 = AdaBoostClassifier()
adaboost3.fit(X_train, y_train)
y_pred = adaboost3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 82.04666666666667


##Gradient Boost

In [133]:
# Create a Gradient Boost
from sklearn.ensemble import GradientBoostingClassifier
grad_boost3 = GradientBoostingClassifier()
grad_boost3.fit(X_train, y_train)
y_pred = grad_boost3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 83.70666666666666


##XG Boost

In [134]:
# Create a XG Boost
import xgboost as xgb
from xgboost import XGBClassifier
xgb_boost3 = xgb.XGBClassifier()
xgb_boost3.fit(X_train, y_train)
y_pred = xgb_boost3.predict(X_test)
y_pred
# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', acc*100)

Test accuracy: 83.74000000000001
